### Read the script

In [ ]:
import os
import mne
from ieeg.navigate import crop_empty_data
from ieeg.io import get_data, raw_from_layout, save_derivative, update

In [ ]:
HOME = os.path.expanduser("~")
LAB_root = os.path.join(HOME, "Box", "CoganLab")
layout = get_data("LexicalDecRepDelay", root=LAB_root)
subjects = layout.get(return_type="id", target="subject")

In [ ]:
subject = 'D0066'
# subject_Tag = 'D107B'
# subject_Tag = 'D0070'

In [ ]:
raw = raw_from_layout(layout, subject=subject, preload=True, extension='.edf')
print(subjects)

### Linear noise filtering
https://ieeg-pipelines.readthedocs.io/en/latest/auto_examples/plot_clean.html

In [ ]:
from ieeg.mt_filter import line_filter

In [ ]:
# n_jobs=-1 means take all the processors, n_jobs=-2 means take all - 1 processors
# Keep all the other parameters not changed
# %% filter data
line_filter(raw, mt_bandwidth=10., n_jobs=-3, copy=False, verbose=10,
            filter_length='700ms', freqs=[60], notch_widths=20)
line_filter(raw, mt_bandwidth=10., n_jobs=-3, copy=False, verbose=10,
            filter_length='20s', freqs=[60, 120, 180, 240],
            notch_widths=20)

## Remove eeg channels
https://ieeg-pipelines.readthedocs.io/en/latest/auto_examples/plot_spectrograms.html
### <span style="color:red">Please pay attention to the section above and see what channels do not have locations when the raw was loaded. Write them down and remove them. They are usually EEG channels</span>

In [ ]:
# Remove EEG channels for D101
found=1
if subject=='D0053':
    eeg_channels_to_exclude=[]
elif subject=='D0054':
    eeg_channels_to_exclude=[]
elif subject=='D0055':
    eeg_channels_to_exclude=[]
elif subject=='D0057':
    eeg_channels_to_exclude=[]
elif subject=='D0059':
    eeg_channels_to_exclude=[]
elif subject=='D0063':
    eeg_channels_to_exclude=[]
elif subject=='D0066':
    eeg_channels_to_exclude=[]
elif subject=='D0069':
    eeg_channels_to_exclude=[]
elif subject=='D0070':
    eeg_channels_to_exclude=[]
elif subject=='D0081':
    eeg_channels_to_exclude=[]
elif subject=='D0094':
    eeg_channels_to_exclude=[]
elif subject=='D0096':
    eeg_channels_to_exclude=[]
elif subject=='D0101':
    eeg_channels_to_exclude = ['Fp1', 'Fp2', 'F3','F4','C3','C4','P3','P4','F7','F8','T3','T4','T5','T6','O1','O2','Fz','Cz','Pz']
elif subject=='D0102':
    eeg_channels_to_exclude = ['T5', 'T6', 'FZ', 'CZ', 'PZ', 'FP1', 'FP2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', '02', 'F7', 'F8', 'T3', 'T4']
elif subject=='D0103':
    eeg_channels_to_exclude = ['FZ', 'CZ', 'PZ', 'F7', 'F8', 'T5', 'T6', 'FP1', 'FP2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'T3', 'T4']
elif subject=='D0107':
    eeg_channels_to_exclude=[]
else:
    print("Subject not found, please check.")
    found=0
    
if found and eeg_channels_to_exclude:
    raw.drop_channels(eeg_channels_to_exclude)
#raw.load_data()

## Save files


In [ ]:
# Check if derivatives folder exists and create if not
bids_root='C:\\Users\\bl314\\Box\\CoganLab\\BIDS-1.0_LexicalDecRepDelay\\BIDS'
if not os.path.exists(os.path.join(bids_root, "derivatives")):
    os.mkdir(os.path.join(bids_root, "derivatives"))
    os.mkdir(os.path.join(bids_root, "derivatives", "a"))
elif not os.path.exists(os.path.join(bids_root, "derivatives", "a")):
    os.mkdir(os.path.join(bids_root, "derivatives", "a"))
save_derivative(raw, layout, "a", True)

## Mark and update outlier channels

In [ ]:
# load data
del raw
raw2 = raw_from_layout(layout.derivatives['derivatives/a'], subject=subject, desc='a',extension='.edf',preload=True)

In [ ]:
from ieeg.navigate import channel_outlier_marker
# mark channel outliers as bad
rawsmall = crop_empty_data(raw2)
rawsmall.info['bads'] = channel_outlier_marker(raw2, 3, 2)
# Exclude bad channels
raw2.info.update(bads=[b for b in eeg_channels_to_exclude if b in raw2.ch_names])
update(raw2, layout, "outlier")
del rawsmall
#raw.load_data()